# 1 . Import modules and dependencies

In [1]:
import csv
import sys
from util import Node, StackFrontier, QueueFrontier
from degrees import person_id_for_name, neighbors_for_person

# 2. Init settings

In [2]:
# Maps names to a set of corresponding person_ids
names = {}
# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}
# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

directory = "small"

# 3. Load data from files into memory

In [3]:
def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

In [4]:
print("Loading data...")
load_data(directory)
print("Data loaded.")

Loading data...
Data loaded.


# 4. Set person 1 & 2

In [5]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [6]:
source = person_id_for_name(input("Name: "))
if source is None:
    sys.exit("Person not found.")
target = person_id_for_name(input("Name: "))
if target is None:
    sys.exit("Person not found.")

Name:  tom hanks
Name:  demi moore


# 5. Find shortest path

In [7]:
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors

In [8]:
def printFailLine(node):
    if node.parent is not None:
        printFailLine(node.parent) 
    print(people[node.state]["name"] + "-->", end =" ")

In [9]:
def printSuccessLine(source, solution):
    print(people[source]["name"] + "-->", end =" ")
    for index, pair in enumerate(solution): 
        if (index != len(solution) - 1):
            print(people[pair[1]]["name"] + "-->", end =" ")
        else:
            print(people[pair[1]]["name"])

In [10]:

start = Node(state = source, parent = None, action = None)
frontier = QueueFrontier()
frontier.add(start)
explored = set()

while True:
    if frontier.empty():
        path = None
        break

    node = frontier.remove()
    if node.state == target:
        solution = []
        while node.parent is not None:
            pair = (node.action, node.state)
            solution.append(pair)
            node = node.parent
        solution.reverse()
        path = solution
        printSuccessLine(source, solution)
        break
        
    printFailLine(node)
    print('X')
    explored.add(node.state)

    for action, state in neighbors_for_person(node.state):
        if not frontier.contains_state(state) and state not in explored:
            child = Node(state=state, parent=node, action=action)
            frontier.add(child)

Tom Hanks--> X
Tom Hanks--> Gary Sinise--> X
Tom Hanks--> Bill Paxton--> X
Tom Hanks--> Robin Wright--> X
Tom Hanks--> Kevin Bacon--> X
Tom Hanks--> Sally Field--> X
Tom Hanks--> Robin Wright--> Chris Sarandon--> X
Tom Hanks--> Robin Wright--> Mandy Patinkin--> X
Tom Hanks--> Robin Wright--> Cary Elwes--> X
Tom Hanks--> Kevin Bacon--> Tom Cruise--> X
Tom Hanks--> Kevin Bacon--> Jack Nicholson--> X
Tom Hanks--> Kevin Bacon--> Demi Moore


# 6. Print result

In [11]:
if path is None:
    print("Not connected.")
else:
    degrees = len(path)
    print(f"{degrees} degrees of separation.")
    path = [(None, source)] + path
    
    for i in range(degrees):
        person1 = people[path[i][1]]["name"]
        person2 = people[path[i + 1][1]]["name"]
        movie = movies[path[i + 1][0]]["title"]
        print(f"{i + 1}: {person1} and {person2} starred in {movie}")

2 degrees of separation.
1: Tom Hanks and Kevin Bacon starred in Apollo 13
2: Kevin Bacon and Demi Moore starred in A Few Good Men
